In [ ]:
#include <cmath>
#include <cassert>
#include <iostream>
#include <random>
#include <numbers>
#include <ctime>
#include "clad/Differentiator/Differentiator.h"
//---------------------
#include <fstream>
auto MY_PI = 3.14159265359;

In [ ]:
namespace clad::custom_derivatives::std {
template <typename T>
CUDA_HOST_DEVICE ValueAndPushforward<T, T> atan2_pushforward(T y, T x, T d_y,
                                                             T d_x) {
  return {::std::atan2(y, x),
          -(y / ((x * x) + (y * y))) * d_x + x / ((x * x) + (y * y)) * d_y};
}

template <typename T, typename U>
CUDA_HOST_DEVICE void atan2_pullback(T y, T x, U d_z, T* d_y, T* d_x) {
  *d_y += x / ((x * x) + (y * y)) * d_z;

  *d_x += -(y / ((x * x) + (y * y))) * d_z;
}

template <typename T>
CUDA_HOST_DEVICE ValueAndPushforward<T, T> acos_pushforward(T x, T d_x) {
  return {::std::acos(x), ((-1) / (::std::sqrt(1 - x * x))) * d_x};
}
}
namespace clad::custom_derivatives
{
    using std::atan2_pushforward;
    using std::acos_pushforward;
}

In [ ]:
double DistanceSquare(double x1, double y1, double z1, double x2, double y2, double z2)
{
    return (x1 - x2) * (x1 - x2) + (y1 - y2) * (y1 - y2) + (z1 - z2) * (z1 - z2);
}

In [ ]:
double Distance(double x1, double y1, double z1, double x2, double y2, double z2)
{
    return std::sqrt(DistanceSquare(x1, y1, z1, x2, y2, z2));
}

In [ ]:
double DistanceSquareA(double v[3], double x2, double y2, double z2)
{
    return DistanceSquare(v[0], v[1], v[2], x2, y2, z2);
}

In [ ]:
double DistanceA(double v[3], double x2, double y2, double z2)
{
    return Distance(v[0], v[1], v[2], x2, y2, z2);
}

In [ ]:
void RotateAlph(double x, double y, double z, double alph, double output[3])
{
    output[0] = x;
    output[1] = y * cos(alph) - z * sin(alph);
    output[2] = y * sin(alph) + z * cos(alph);
}

In [ ]:
void RotateBet(double x, double y, double z, double bet, double output[3])
{
    output[0] = x * cos(bet) + z * sin(bet);
    output[1] = y;
    output[2] = -x * sin(bet) + z * cos(bet);
}

In [ ]:
void Rotate(double x, double y, double z, double alph, double bet, double output[3])
{
    double point[3];
    RotateAlph(x, y, z, alph, point);
    RotateBet(point[0], point[1], point[2], bet, output);
}

In [ ]:
void UnRotate(double x, double y, double z, double alph, double bet, double output[3])
{
    double point[3];
    RotateBet(x, y, z, -bet, point);
    RotateAlph(point[0], point[1], point[2], -alph, output);
}

In [ ]:
double EvaluateSinPlusLin(double A, double B, double C, double D, double x)
{
    /*When this equation is equal to zero, the distance between the point and the helix is the shortest.*/
    return A * std::sin(x + B) + C * x + D;
}

In [ ]:
// A sin (x + B) + C x + D = 0
double SolveSinPlusLin(double A, double B, double C, double D, double mi, double ma)
{
    /*Binary search to determine x, with which EvaluateSinPlusLin equation equals zero.*/
    for (int i = 0; i < 100; i++)
    {
        double mid = (mi + ma) / 2;
        double vmi = EvaluateSinPlusLin(A, B, C, D, mi);
        double vmid = EvaluateSinPlusLin(A, B, C, D, mid);
        double vma = EvaluateSinPlusLin(A, B, C, D, ma);

        if (vmi < 0 and 0 < vmid)
        {
            ma = mid;
        }
        else if (vmid < 0 and 0 < vma)
        {
            mi = mid;
        }
        else if (vmid < 0 and 0 < vmi)
        {
            ma = mid;
        }
        else if (vma < 0 and 0 < vmid)
        {
            mi = mid;
        }
        else
        {
            break;
            mi = mid;
        }
    }

    double x = (mi + ma) / 2;
    return x;
}

In [ ]:
double NextValPiK(double offs, double x)
{

    if (x < 0)
    {
        double v = -NextValPiK(-offs, -x) + 2 * MY_PI;
        return v > x ? v : v + 2 * MY_PI;
    }

    double kie = std::floor(x / 2 / MY_PI);

    for (int i = -2; i <= 2; i++)
    {
        double v = (kie + i) * 2 * MY_PI + offs;

        if (v > x)
        {
            return v;
        }
    }

    return 1000000000;
}

In [ ]:
// A cos(x + B) + C = 0
double NextSinPlusInflection(double A, double B, double C, double x)
{
    /* Identifies the next inflection point of the sine curve.*/
    // cos(x + B) = -C / A
    if (-C / A >= -1 && -C / A <= 1)
    {
        double inv = std::acos(-C / A);
        return std::min(NextValPiK(inv - B, x), NextValPiK(-inv - B, x));
    }
    else
    {
        return 1000000000;
    }
}

In [ ]:
void HelixPoint(double a, double b, double c, double d, double alph, double bet, double t, double output[3])
{
    /*Describe a point on a helix in the Cartesian coordinate system.*/
    double x = a * (c + std::cos(t));
    double y = a * (d + std::sin(t));
    double z = a * b * t;
    output[0] = x;
    output[1] = y;
    output[2] = z;
    Rotate(x, y, z, alph, bet, output);
}

In [ ]:
double HelixClosestTime(double a, double b, double c, double d, double alph, double bet, double x, double y, double z)
{
    /*Calculate t, during which a helix with given params is the closest to a given point.*/
    double point[3];
    UnRotate(x, y, z, alph, bet, point);
    point[0] /= a;
    point[1] /= a;
    point[2] /= a;
    point[0] -= c;
    point[1] -= d;
    double A = std::sqrt(point[0] * point[0] + point[1] * point[1]);
    double B = std::atan2(-point[1], point[0]);
    double C = b * b;
    double D = -point[2] * b;

    double mi = point[2] / b - MY_PI;
    double ma = point[2] / b + MY_PI;
    double t1 = SolveSinPlusLin(A, B, C, D, mi, ma);

    double ans = t1;
    HelixPoint(a, b, c, d, alph, bet, ans, point);
    double dist = DistanceSquareA(point, x, y, z);

    for (double t = mi; t < ma; t = t)
    {
        double ttt = NextSinPlusInflection(A, B, C, t);

        if (ttt == t)
        {
            break;
        }

        double cur = SolveSinPlusLin(A, B, C, D, t, ttt);
        t = ttt;
        HelixPoint(a, b, c, d, alph, bet, cur, point);
        double dist2 = DistanceSquareA(point, x, y, z);

        if (dist2 < dist)
        {
            dist = dist2;
            ans = cur;
        }
    }

    return ans;
}

In [ ]:
// All the matrices are written as 1D arrays!
void MatrixMultiply(double *a, double *b, int arows, int acols, int bcols, double *output)
{
    for (int i = 0; i < arows; i++)
    {
        for (int j = 0; j < bcols; j++)
        {
            double sum = 0;
            for (int k = 0; k < acols; k++)
                sum = sum + a[i * acols + k] * b[k * bcols + j];
            output[i * bcols + j] = sum;
        }
    }
}

In [ ]:
void Transpose(double *input, int rows, int cols, double *output)
{
    for (int i = 0; i < rows; ++i)
    {
        for (int j = 0; j < cols; ++j)
        {
            int i_input = i * cols + j;

            int i_output = j * rows + i;

            output[i_output] = input[i_input];
        }
    }
}

In [ ]:
void DiagOfSquareM(double *input, int height, double *diag)
{
    // Works for square matrices only
    for (int i = 0; i < height * height; i++)
    {
        diag[i] = 0;
    }
    for (int i = 0; i < height; i++)
    {
        diag[i * height + i] = input[i * height + i];
    }
}

In [ ]:
void ScalarMultiply(double *matrix, int rows, int cols, double number, double *output)
{
    for (int i = 0; i < rows; i++)
    {
        for (int j = 0; j < cols; j++)
        {
            output[i * cols + j] = number * matrix[i * cols + j];
        }
    }
}


In [ ]:
void CopyMatrix(double *matrix, int size, double *output)
{
    for (int i = 0; i < size; i++)
    {
        output[i] = matrix[i];
    }
}

In [ ]:
void AddMatrices(double *a, double *b, int rows, int cols, double *output)
{
    for (int i = 0; i < rows; i++)
    {
        for (int j = 0; j < cols; j++)
        {
            output[i * cols + j] = a[i * cols + j] + b[i * cols + j];
        }
    }
}

In [ ]:
void swap_row(double *matrix, int size, int i, int j)
{
    // 6x6
    for (int k = 0; k < size; k++)
    {
        double temp = matrix[i * size + k];
        matrix[i * size + k] = matrix[j * size + k];
        matrix[j * size + k] = temp;
    }
}

In [ ]:
void ForwardElim(double *input, int size, double *res, double *output)
{
    for (int i = 0; i < size; ++i)
    {
        for (int j = 0; j < size; ++j)
        {
            output[i * size + j] = input[i * size + j];
        }
    }
    for (int i = 0; i < size; i++)
    {
        int i_max = i;
        double v_max = output[i_max * size + i];

        for (int j = i + 1; j < size; j++)
            if (std::abs(output[j * size + i]) > std::abs(v_max) && output[j * size + i] != 0)
                v_max = output[j * size + i], i_max = j;
        if (i_max != i)
        {
            swap_row(output, size, i, i_max);
            double temp = res[i];
            res[i] = res[i_max];
            res[i_max] = temp;
        }

        if (output[i * size + i] == 0.0)
        {
            std::cerr << "Mathematical Error!";
            std::cerr << "Input that caused the error is:";
            for (int i = 0; i < size; i++)
            {
                for (int j = 0; j < size; j++)
                {
                    std::cerr << output[i * size + j] << " ";
                }
                std::cerr << std::endl;
            }
        }
        for (int j = i + 1; j < size; j++)
        {
            double ratio = output[j * size + i] / output[i * size + i];

            for (int k = 0; k < size; k++)
            {
                output[j * size + k] = output[j * size + k] - ratio * output[i * size + k];
                if (std::abs(output[j * size + k]) <= 1e-15)
                {
                    output[j * size + k] = 0;
                }
            }
            res[j] = res[j] - ratio * res[i];
            if (std::abs(res[j]) <= 1e-15)
            {
                res[j] = 0;
            }
        }
    }

    std::cerr << "Forward elimination results:" << std::endl;
    std::cerr << "Left side:" << std::endl;
    for (int j = 0; j < size; j++)
    {
        for (int k = 0; k < size; k++)
        {
            std::cerr << output[j * size + k] << " ";
        }
        std::cerr << std::endl;
    }
    std::cerr << "Right side:" << std::endl;
    for (int k = 0; k < size; k++)
    {
        std::cerr << res[k] << " ";
    }
    std::cerr << std::endl;
}

In [ ]:
void BackSub(double *input, int size, double *right_side, double *results)
{
    /*Back substitution and the result of Gaussian elimination*/
    for (int i = (size - 1); i > -1; i--)
    {
        results[i] = right_side[i];
        for (int j = (size - 1); j > i; j--)
        {
            results[i] -= input[i * size + j] * results[j];
        }
        results[i] /= input[i * size + i];
        if (std::abs(results[i]) <= 1e-15)
        {
            results[i] = 0;
        }
    }
}

In [ ]:
void CheckSolution(double *input, int size, double *right_side, double *results)
{
    for (int i = 0; i < size; i++)
    {
        double sum = 0;
        for (int j = 0; j < size; j++)
        {
            sum += input[i * size + j] * results[j];
        }
        if (std::abs(sum - right_side[i]) >= 1e-5)
        {
            std::cerr << "Wrong solution " << sum << " " << right_side[i] << std::endl;
            std::abort();
        }
    }
}

In [ ]:
void PrintMatrix(std::string name, double *matrix, int rows, int cols)
{
    std::cerr << name << std::endl;
    for (int i = 0; i < rows; i++)
    {
        for (int j = 0; j < cols; j++)
        {
            std::cerr << matrix[i * cols + j] << " ";
        }
        std::cerr << std::endl;
    }
}

In [ ]:
double DistanceToPoint(double a, double b, double c, double d, double alph, double bet, double x, double y, double z)
{
    /*Calculate the distance to a single point. */
    double t = HelixClosestTime(a, b, c, d, alph, bet, x, y, z);
    double output[3];
    HelixPoint(a, b, c, d, alph, bet, t, output);
    double dist = DistanceA(output, x, y, z);
    dist += 0.001 * ((a * a) + (b * b) + (c * c) + (d * d) + (alph * alph) + (bet * bet));

    return dist;
}

In [ ]:
void DistancesToAllPoints(double *points, int nr_of_points, double a, double b, double c, double d, double alph, double bet, double *dist)
{
    /*Calculate the distances to all points. */
    int n = 0;
    for (int i = 0; i < nr_of_points; i++)
    {
        double x = points[i * 3];
        double y = points[i * 3 + 1];
        double z = points[i * 3 + 2];
        dist[n] = DistanceToPoint(a, b, c, d, alph, bet, x, y, z);
        n++;
    }
}

In [ ]:
double SquareErr(double *points, int nr_of_points, double a, double b, double c, double d, double alph, double bet)
{
    /*Calculate the residual sum of squares. */
    double dist;
    double square_err = 0;
    for (int i = 0; i < nr_of_points; i++)
    {
        double x = points[i * 3];
        double y = points[i * 3 + 1];
        double z = points[i * 3 + 2];
        dist = DistanceToPoint(a, b, c, d, alph, bet, x, y, z);
        square_err += (dist * dist);
    }
    return square_err;
}

In [ ]:
void Points(double *points, int nr_of_points)
{
    /*Generate points on a helix with given params. */
    double a = 5.2122, b = 2, c = 10.835, d = 17.07055, alph = -3.60384, bet = 1.13255;
    double t = 0;
    for (int i = 0; i < nr_of_points; i++)
    {
        t += 0.1;
        double output[3];
        HelixPoint(a, b, c, d, alph, bet, t, output);
        double x = output[0], y = output[1], z = output[2];
        points[i * 3] = x;
        points[i * 3 + 1] = y;
        points[i * 3 + 2] = z;
    }
    std::cerr << "Target square err:\n";
    double square_err = SquareErr(points, nr_of_points, a, b, c, d, alph, bet);
    std::cerr << square_err << std::endl;
}


In [ ]:
void GenerateFlawedPoints(int nr_of_points, double a, double b, double c, double d, double alph, double bet, double *points)
{
    /*Generate points on a helix with given params but add noise. */
    auto seed = time(nullptr);
    std::mt19937_64 rng(seed);
    std::uniform_real_distribution<double> uniform(0, 1);
    double output[3];
    double t = 0;
    for (int i = 0; i < nr_of_points; i++)
    {
        t += 0.1;
        HelixPoint(a, b, c, d, alph, bet, t, output);
        points[i * 3] = output[0] + uniform(rng);
        points[i * 3 + 1] = output[1] + uniform(rng);
        points[i * 3 + 2] = output[2] + uniform(rng);
    }
}


In [ ]:
void Jacobian(double *points, int nr_of_points, double a, double b, double c, double d, double alph, double bet, double *Jacobian)
{
    /*Construct the nr_of_points x 6 Jacobian.*/
    auto dist_grad = clad::gradient(DistanceToPoint, "a, b, c, d, alph, bet");
    for (int i = 0; i < nr_of_points; i++)
    {
        double x = points[i * 3];
        double y = points[i * 3 + 1];
        double z = points[i * 3 + 2];
        double output[3];
        double da = 0, db = 0, dc = 0, dd = 0, dalph = 0, dbet = 0;
        dist_grad.execute(a, b, c, d, alph, bet, x, y, z, &da, &db, &dc, &dd, &dalph, &dbet);
        Jacobian[i * 6] = da;
        Jacobian[i * 6 + 1] = db;
        Jacobian[i * 6 + 2] = dc;
        Jacobian[i * 6 + 3] = dd;
        Jacobian[i * 6 + 4] = dalph;
        Jacobian[i * 6 + 5] = dbet;
    }
}

In [ ]:
double Lambda(double *points, int nr_of_points, double &a, double &b, double &c, double &d, double &alph, double &bet, double lambda, double &old_square_err, double *results)
{
    /*Calculate the damping coefficient lambda for the next iteration of the LevenbergMarquardt function.*/
    double new_lambda;
    double square_err = SquareErr(points, nr_of_points, a + results[0], b + results[1], c + results[2], d + results[3], alph + results[4], bet + results[5]);
    std::cerr << "SQUARE ERR " << square_err << std::endl;
    if ((square_err >= old_square_err) && (lambda < 1000))
        new_lambda = lambda * 10;
    else
    {
        std::cerr << "IMPROVEMENTS!";
        a += results[0];
        b += results[1];
        c += results[2];
        d += results[3];
        alph += results[4];
        bet += results[5];
        new_lambda = lambda / 10;
        old_square_err = square_err;
    }
    return new_lambda;
}


In [ ]:
void LevenbergMarquardt(double *points, int nr_of_points, double true_b)
{
    /*Use the Levenberg-Marquardt algorithm to fit a helix on a given set of points. Currently produces all of the parameters of the helix, except b.*/
    double a = 6.2122, b = 0.1, c = 1.9835, d = 1.707055, alph = -3.60384, bet = 1.13255; // currently breaks if the parameters are exact as the ones used for (noise free) generated points

    int diff_params = 6;
    double lambda = 1;
    double lambda_change = 1;
    double old_square_err;
    double jacobian[nr_of_points * diff_params];
    double tjacobian[diff_params * nr_of_points];
    double tjj[diff_params * diff_params];
    double results[diff_params];
    {
        double dist[nr_of_points];
        DistancesToAllPoints(points, nr_of_points, a, b, c, d, alph, bet, dist);
        old_square_err = 0;
        for (int i = 0; i < nr_of_points; i++)
        {
            old_square_err += (dist[i] * dist[i]);
        }
    }

    for (int i = 0; i < 2000; i++)
    {

        Jacobian(points, nr_of_points, a, b, c, d, alph, bet, jacobian);

        Transpose(jacobian, nr_of_points, diff_params, tjacobian);

        MatrixMultiply(tjacobian, jacobian, diff_params, nr_of_points, diff_params, tjj);

        double diag[diff_params * diff_params];
        DiagOfSquareM(tjj, diff_params, diag);

        double identity[diff_params * diff_params];
        ScalarMultiply(diag, diff_params, diff_params, lambda, identity);
        double left_side[diff_params * diff_params];
        AddMatrices(tjj, identity, diff_params, diff_params, left_side);
        double dist[nr_of_points];
        DistancesToAllPoints(points, nr_of_points, a, b, c, d, alph, bet, dist);
        double right_side[diff_params * 1];
        MatrixMultiply(tjacobian, dist, diff_params, nr_of_points, 1, right_side);
        ScalarMultiply(right_side, 1, diff_params, -1, right_side);

        // left side is 6x6, right side is 6x1, so h is 6x1.
        double forward_elim[diff_params * diff_params];
        double unchanged_rs[diff_params];
        CopyMatrix(right_side, diff_params, unchanged_rs);
        ForwardElim(left_side, diff_params, right_side, forward_elim);
        BackSub(forward_elim, diff_params, right_side, results);
        CheckSolution(left_side, diff_params, unchanged_rs, results);
        lambda = Lambda(points, nr_of_points, a, b, c, d, alph, bet, lambda, old_square_err, results);
        std::cerr << "New params: " << a << " " << b << " " << c << " " << d << " " << alph << " " << bet << " ";
        std::cerr << "lambda: " << lambda << " squares distance: " << old_square_err << std::endl;
    }

    double t = 0;
    for (int i = 0; i < nr_of_points; i++)
    {
        t += 0.1;
        double output[3];
        HelixPoint(a, true_b, c, d, alph, bet, t, output);
        double x = output[0], y = output[1], z = output[2];

        std::cout << x << " " << y << " " << z << "\n";
    }
    std::cout << "end\n";
}

In [ ]:
int main()
{
    int nr_of_points = 200;
    double points[nr_of_points * 3];
    double a = 5.2122, b = 2, c = 10.835, d = 17.07055, alph = -3.60384, bet = 1.13255;
    GenerateFlawedPoints(nr_of_points, a, b, c, d, alph, bet, points);
    LevenbergMarquardt(points, nr_of_points, b);
    for (int i = 0; i < nr_of_points; i++)
    {
        std::cout << points[i * 3 + 0] << " " << points[i * 3 + 1] << " " << points[i * 3 + 2] << "\n";
    }
    std::cout << "end\n";
}